In [ ]:
import pandas as pd
from pathlib import Path

datadir = Path('/work/data/mushtaq/cellIQ 30.06.2020 20ulseeding a2WT a3TRI b2Afa b3Magi c2Occ c3LSR/output')

wells = datadir.glob('Well*')

# this will contain all data
df = pd.DataFrame()

for w in wells:
    print(w)
    files = (w / "ar").glob("*.csv")
    for f in files:
        #print(f)
        tmp = pd.read_csv(f)
        
        # add a column that shows which well the numbers are from
        tmp["Well"] = w.stem
        
        # add a column that shows the timepoint
        timestr = f.stem.replace("_segmentation_results","")
        timestr = timestr.split('_')
        timestr = "_".join(timestr[:5])
        tmp['Timepoint'] = timestr
        tmp['Timepoint'] = pd.to_datetime(tmp['Timepoint'], format='%Y_%m_%d_%H_%M')
        
        # add to the main dataframe
        df = df.append(tmp, ignore_index=True)



In [ ]:
# add a column that show time from start of experiment
start_time = df['Timepoint'].min()
df['T'] = df['Timepoint'] - start_time
df.head()

In [ ]:
# write data to one .csv
df.to_csv(datadir / 'all_measurements_in_one_file.csv', index=False)


In [ ]:
# combine the AR data in another dataframe, take mean over timepoint
mean_ar = pd.pivot_table(df, index='T', columns='Well', values='AR', aggfunc='mean')

# write mean AR data in one .csv
mean_ar.to_csv(datadir / 'mean_ar_all_in_one_file.csv', index=False)

mean_ar.head()

In [ ]:
import matplotlib.pyplot as plt

# prepare to plot data
fig,ax = plt.subplots(figsize=(25,15))

# loop over wells
for c in mean_ar.columns:
    print(c)
    
    # The NaN values on mean_ar cause problems for plotting.
    # Workaround is to... create a new dataframe, keep
    keep = [c]
    # ...make copy of the data
    tmp = mean_ar.copy()
    # ...keep only the interesting column (well)
    tmp = tmp.drop(tmp.columns.difference(keep), axis=1) 
    # ...drop NaN values
    tmp = tmp.dropna()

    # plot the data
    tmp.plot(ax=ax)

# set y axis range
plt.ylim(1,5)
# set y axis label
plt.ylabel('Mean AR')

tmp.head()

In [ ]:
df.AR.min()

In [ ]:
df.AR.max()